<a href="https://colab.research.google.com/github/kfl15/NLP/blob/main/Word_Embedding_For_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.11.0


In [3]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3N5XJ6/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3N5XJ6/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3N5XJ6/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Segregating training and testing sets


In [4]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [5]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [6]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [7]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16
max_length = 150
trunc_type='post'
oov_tok = "<oov>"


In [8]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   11   26   75  571
    6  805 2354  313  106   19   12    7  629  686    6    4 2219    5
  181  584   64 1454  110 2263    3 3951   21    2    1    3  258   41
 4677    4  174  188   21  

## Define the Neural Network with Embedding layer

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 6)                 14406     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 174,413
Trainable params: 174,413
Non-trainable params: 0
_________________________________________________________________


## Model Training

In [11]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 47s 54ms/step - loss: 0.4817 - accuracy: 0.7515 - val_loss: 0.3175 - val_accuracy: 0.8639
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2241 - accuracy: 0.9161 - val_loss: 0.3316 - val_accuracy: 0.8593
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0853 - accuracy: 0.9793 - val_loss: 0.3982 - val_accuracy: 0.8472
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0213 - accuracy: 0.9974 - val_loss: 0.4695 - val_accuracy: 0.8486
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0056 - accuracy: 0.9996 - val_loss: 0.5280 - val_accuracy: 0.8472
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.5723 - val_accuracy: 0.8494
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 8.2130e-04 - accuracy: 1.0000 - val_loss: 0.6144 - val_accuracy: 0.84

## Deriving weights from the embedding layer

In [13]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights)

(10000, 16)
[[ 0.04846428 -0.01671404 -0.0440569  ...  0.00831733  0.01184543
  -0.00389499]
 [ 0.02468192 -0.0398302  -0.06342699 ... -0.00236944 -0.02130353
  -0.02141498]
 [ 0.07648127 -0.09101198 -0.01393503 ...  0.02632445 -0.0596807
  -0.02624289]
 ...
 [-0.02762834  0.0080433  -0.09931878 ...  0.01595287  0.02550207
  -0.10695866]
 [ 0.06604881 -0.08523058  0.04877773 ...  0.05910269  0.00874028
  -0.1384512 ]
 [-0.13180496 -0.10667649 -0.01912361 ...  0.05283969  0.09962321
  -0.12303159]]


## Downloading the vectors and metadata

In [14]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')


##write each word and its corresponding embedding
for index in range(1, vocab_size):
  word = reverse_word_index[index]  # flipping the key-value in word_index
  embeddings = weights[index]
  meta.write(word + "\n")
  vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [15]:
##download the written files to local machine
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vectors.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>